# Imports

In [ ]:
import pandas as pd
import os, gc
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
data_root = 'datawarehouse/' # '../../../data/ivy-hip-fasa/datawarehouse/'
filenames = os.listdir(data_root)
print(filenames)

# Load Files

## ACADEMIC_PROGRAM_FACT

In [ ]:
academic_program_fact = pd.read_csv(data_root + 'ACADEMIC_PROGRAM_FACT.csv')
column_mask = [col for col in academic_program_fact.columns if 'ETL' not in col]
academic_program_fact= academic_program_fact[column_mask]
academic_program_fact.head(3)

In [ ]:
academic_program_fact['ACADEMIC_PROGRAM_STATUS'].value_counts(normalize=True)

In [4]:
# we only need active students
# commented out for now, since students who are not active now, might get aid when they were active
academic_program_fact = academic_program_fact[
    academic_program_fact['ACADEMIC_PROGRAM_STATUS']=='Active in Program'
]

# next we don't need this single valued column anymore
academic_program_fact.drop(
    columns=[
        'ACADEMIC_PROGRAM_STATUS', 'ACADEMIC_PROGRAM_FACT_ID', 
        'ADMIT_TYPE_DIM_ID', 'ADMIT_TERM_DIM_ID'
    ], inplace=True
)

In [ ]:
academic_program_fact.drop_duplicates(
    subset=['ACADEMIC_PROGRAM_DIM_ID', 'PARTY_DIM_ID'],
    inplace=True
)
print(academic_program_fact.shape)
academic_program_fact.head(3)

## ACADEMIC_PLAN_DIM

In [ ]:
academic_plan_dim = pd.read_csv(data_root + 'ACADEMIC_PLAN_DIM.csv')
column_mask = [col for col in academic_plan_dim.columns if 'ETL' not in col]
academic_plan_dim= academic_plan_dim[column_mask]
academic_plan_dim.head(3)

In [7]:
# we are only evaluating undergrads
academic_plan_dim = academic_plan_dim[academic_plan_dim['ACADEMIC_CAREER']=='UGRD']

# not necessary if undergrad is the only value
academic_plan_dim.drop(
    columns=['ACADEMIC_CAREER', 'ACADEMIC_CAREER_DESC'], 
    inplace=True
)

## ACADEMIC_PROGRAM_DIM

In [ ]:
academic_program_dim = pd.read_csv(data_root + 'ACADEMIC_PROGRAM_DIM.csv')
column_mask = [col for col in academic_program_dim.columns if 'ETL' not in col]
academic_program_dim = academic_program_dim[column_mask]

academic_program_dim.dropna(inplace=True)
academic_program_dim['ACADEMIC_PROGRAM_DESC'] = academic_program_dim[
    'ACADEMIC_PROGRAM_DESC'].apply(lambda x: x.replace(' Undergraduate', ''))
academic_program_dim.head(3)

In [9]:
academic_program_dim.drop(
    columns='ACADEMIC_CAREER_DESC', inplace=True
)

## DISBURSEMENT

In [ ]:
disbursement = pd.read_csv(data_root + 'DISBURSEMENT_UGRD_20142015.csv')
disbursement.head(3)

In [ ]:
disbursement.groupby('AID_YEAR')['OFFER_BALANCE'].sum().reset_index()

In [ ]:
# incomplete info for academic year 2023 or air year 2024
disbursement = disbursement[disbursement['AID_YEAR']<=2024].reset_index(drop=True)

# to identify each disbursement uniquely. 
disbursement = disbursement.reset_index(names='DISBURSEMENT_ID')
disbursement.drop(
    columns=[
        'ACADEMIC_CAREER', 'ITEM_TYPE', 
        'AID_YEAR_DESC', 'ITEM_TYPE_DESC', 
        'DISBURSEMENT_DESC'], 
    inplace=True
)
disbursement.groupby('AID_YEAR')['OFFER_BALANCE'].sum()

### Access UVA

In [13]:
access_uva = pd.read_csv('Access UVA.csv', header=0)

In [ ]:
columns = ['UVA_ACCESS', 'SCHEV', 'Need based']
access_uva = access_uva[columns]
result = {
    col:[] for col in columns
}

for row in access_uva.values:
    category, schev, need_based = row
    
    scheves = [i.strip() for i in schev.split(',')]
    N = len(scheves)
    
    result[columns[0]].extend([category]*N)
    result[columns[1]].extend(scheves)
    result[columns[2]].extend([need_based]*N)
    
result = pd.DataFrame(result)
result.rename({'SCHEV': 'REPORT_CODE'}, axis=1, inplace=1)
result.head(3)

In [ ]:
print(set(disbursement['REPORT_CODE'].unique()) & set(result['REPORT_CODE'].unique()))
print(set(disbursement['REPORT_CODE'].unique()) - set(result['REPORT_CODE'].unique()))
print(set(result['REPORT_CODE'].unique()) - set(disbursement['REPORT_CODE'].unique()))

In [16]:
# inner join drops non-access uva reports . e.g. XXX
disbursement = disbursement.merge(result, on='REPORT_CODE', how='inner')

# disbursement['Legacy Category order of money'].fillna('Non Access UVA', inplace=True)
# disbursement['Need based'].fillna('NA', inplace=True)

## IAS_STUDENT_INFO_V_UGRD

In [ ]:
ias_student_info = pd.read_csv(data_root + 'IAS_STUDENT_INFO_V_UGRD.csv')
ias_student_info.head(3)

## INCOME_FLAGS_UGRD_20142015

In [ ]:
ias_income_flags = pd.read_csv(data_root + 'INCOME_FLAGS_UGRD_20142015.csv')
ias_income_flags.head(3)

In [ ]:
ias_income_flags['LOW_INCOME_FLAG'].value_counts(normalize=True)

In [20]:
# dataset has only one value 'Daily'
# DEPENDENCY_STATUS and NUM_IN_HOUSEHOLD are majorly missing , 59%
ias_income_flags.drop(columns=['DATASET', 'DEPENDENCY_STATUS', 'NUM_IN_HOUSEHOLD', 'POVERTY_INDEX_YEAR'], inplace=True)

## PS_STDNT_AID

In [ ]:
ps_student_aid = pd.read_csv(data_root + 'PS_STDNT_AID.csv')

# INSTITUTION is always UVA01
ps_student_aid.drop(columns=['INSTITUTION'], inplace=True) 
ps_student_aid.head(3)

In [ ]:
ps_student_aid[ps_student_aid.isna().any(axis=1)].shape

In [23]:
# droping NaNs
ps_student_aid.dropna(inplace=True)

# the data after 2023 is future data, hence changeable
# disbursement file start from 2014
# ps_student_aid = ps_student_aid[
#     (ps_student_aid['AID_YEAR']>=2014) &
#     (ps_student_aid['AID_YEAR']<=2023)
# ]
# surprisingly this id is float in this file. Which shouldn't be the case
ps_student_aid['PARTY_DIM_ID'] = ps_student_aid['PARTY_DIM_ID'].astype(int)

In [ ]:
ps_student_aid.groupby('AID_YEAR')['INST_NEED'].sum().reset_index().tail(5)

## TERM_DIM

In [ ]:
term_dim = pd.read_csv(data_root + 'TERM_DIM.csv')
column_mask = [col for col in term_dim.columns if 'ETL' not in col]
term_dim= term_dim[column_mask]
term_dim.head(3)

In [ ]:
# a lot of missing values
term_dim[term_dim.isna().any(axis=1)].shape

In [ ]:
# print(f'Shape before {term_dim.shape}')
# # only the first two rows are NaN
# term_dim.dropna(inplace=True)
# print(f'Shape after dropping NaNs {term_dim.shape}')

# print('We are only considering Fall terms for now')
# term_dim = term_dim[term_dim['TERM_TYPE']=='Fall']

print('Keeping only undergrad students')
term_dim = term_dim[term_dim['ACADEMIC_CAREER']=='UGRD']
# we can safely drop term type now

term_dim['CALENDAR_YEAR'] = term_dim['CALENDAR_YEAR'].astype(int)
# print('Dropping terms after calendar year 2023.')
# term_dim = term_dim[term_dim['CALENDAR_YEAR']<2024]

drop_columns = ['TERM_DESC', 'ACADEMIC_CAREER', 'ACADEMIC_YEAR']
print(f'Dropping columns {drop_columns}')
term_dim.drop(columns=drop_columns, inplace=True)

term_dim.head(3)

In [ ]:
# TERM_DIM_ID is the primary key here and unique to a term regardless of the year
print(term_dim['TERM_DIM_ID'].nunique() == term_dim.shape[0])

# disbursement file only has complete year info from calendar year 2015 to 2023
# term_dim[term_dim['TERM_DIM_ID'].isin(disbursement['TERM_DIM_ID'])]

## STUDENT_TERM_FACT

In [ ]:
student_term_fact = pd.read_csv(data_root + 'STUDENT_TERM_UGRD_20142015.csv')

column_mask = [col for col in student_term_fact.columns if 'ETL' not in col]
student_term_fact = student_term_fact[column_mask]
student_term_fact.head(3)

In [ ]:
# almost always `Y`, so can be safely dropped
student_term_fact['FIN_AID_PROG_ELIGIBILITY_FLAG'].value_counts(normalize=True)

In [ ]:
student_term_fact = student_term_fact[
    student_term_fact['BILLING_CAREER']=='UGRD'
]

min_aid_year = disbursement['AID_YEAR'].min()
print(min_aid_year)

student_term_fact = student_term_fact[
    student_term_fact['FINANCIAL_AID_ACADEMIC_YEAR'] >= min_aid_year
]

student_term_fact['ACADEMIC_LEVEL_TERM_START'].value_counts(normalize=True)

In [ ]:
# these outliers are rare
print(f'Dropping non set and post-bacc undergrads')
student_term_fact = student_term_fact[
    ~student_term_fact['ACADEMIC_LEVEL_TERM_START'].isin(
        ['Not Set', 'Post-Bacc Undergraduate']
    )
]

student_term_fact.rename({
    'PRIM_ACADEMIC_PROGRAM_DIM_ID': 'PROGRAM_DIM_ID',
    'FINANCIAL_AID_ACADEMIC_YEAR': 'AID_YEAR'
}, axis=1, inplace=True)

In [ ]:
# few nan values which can be safely dropped
student_term_fact['FIN_AID_FED_RES'].value_counts(normalize=True)

In [ ]:
# few outliers with None and Y values. can safely drop them
student_term_fact = student_term_fact[student_term_fact['FIN_AID_FED_RES'].isin(['V', 'N'])]

student_term_fact[student_term_fact['PARTY_DIM_ID']==2048271]

In [35]:
# student_term_fact.groupby(['AID_YEAR', 'TERM_DIM_ID', 'PARTY_DIM_ID']).size().reset_index()
# there were two entries for a party in each aid year and term id, so we need to drop them
student_term_fact.drop_duplicates(
    subset='STUDENT_TERM_FACT_ID',
    inplace=True
)

In [ ]:
# mostly approved full-time
student_term_fact['APPROVED_ACADEMIC_LOAD'].value_counts(normalize=True)

In [37]:
student_term_fact.drop(
    columns=[
        'STUDENT_TERM_FACT_ID', 'BILLING_CAREER', 'FIRST_TERM_IN_CAREER_DIM_ID',
        'FINANCIAL_AID_LOAD', 'APPROVED_ACADEMIC_LOAD', # these two loads are almost same 
        'FIN_AID_PROG_ELIGIBILITY_FLAG'],
    inplace=True
)

# Merge

## Utils

In [38]:
def merge(left:pd.DataFrame, right:pd.DataFrame, key:list|str=None, how='inner'):
    common = set(left.columns) & set(right.columns)
    
    # if no key is provided, assume the common columns that 
    # ends with '_ID' or  '_YEAR' are the common keys
    if key is None:
        key = [
            column for column in common 
                if left[column].dtype in [object, str] or column.endswith('_YEAR') or column.endswith('_ID')
        ]
        if len(key)>0:
            print(f'Found key {key}.')
        else:
            print('Error ! no common key IDs found. Returning None.')
            return None
    
    if type(key) != list: 
        common = [col for col in common if col != key]
    else:
        common = [col for col in common if col not in key]
        
    merged_data = left.merge(right.drop(columns=common), on = key, how=how)
    print(f'Shape: left {left.shape}, right {right.shape}, merged {merged_data.shape}.\n')
    
    return merged_data

## Disbursement

In [ ]:
# aid file doesn't have info for all students in the disbursement file, hence left join
df = merge(disbursement, ps_student_aid, how='left')

# some terms of 2024 will be dropped
df = merge(df, term_dim)

# income flag doesn't have info for all students in the disbursement file, hence left join
df = merge(df, ias_income_flags, how='left')

# not set and post-bacc students will be dropped, hence file size will decrease
df = merge(df, student_term_fact)

academic_program_dim.rename(
    {'ACADEMIC_PROGRAM_DIM_ID': 'PROGRAM_DIM_ID'}, 
    axis=1, inplace=True
)
# added ACADEMIC_PROGRAM_DESC
df = merge(df, academic_program_dim)

## Fill missing values

In [40]:
def missing_percentage(df, nonzero_only=True):
    results = df.isnull().mean().round(4).mul(100).sort_values(ascending=False)
    if nonzero_only:
        results = results[results>0]
    
    return results

In [ ]:
# check for any missing values
missing_df = missing_percentage(df)
missing_df

In [ ]:
print('Imputing missing values\n')

for col in missing_df.index:
    column_type = df[col].dtype
    isnumeric = df[col].dtype in [int, float]
    
    print(f'Column {col}, type {df[col].dtype}, is numeric {isnumeric}')
    
    if isnumeric:
        print('Filling with zero')
        df[col] = df[col].fillna(0)
    else:
        most_freq = df[col].value_counts().reset_index().iloc[0,0]
        print(f'Filling with the most freq item: {most_freq}')
        df[col] =df[col].fillna(most_freq)
    print()
    
print(df.shape)
df.dropna(inplace=True)
print(df.shape)

In [ ]:
df.head(3)

## Summary statistics

In [ ]:
# funds are mostly disbursed in Fall and Spring sessions
df['TERM_TYPE'].value_counts(normalize=True)

In [ ]:
# funding freq is balanced despite level difference
df['ACADEMIC_LEVEL_TERM_START'].value_counts(normalize=True)

# Aggregate

In [ ]:
time_column = 'AID_YEAR' # 'CALENDAR_YEAR'
print(sorted(df.columns))

## Group ids 

In [47]:
groupby_key = [
    'ACADEMIC_LEVEL_TERM_START',
    'ACADEMIC_PLAN',
    'ACADEMIC_PROGRAM_DESC',
    'FIN_AID_FED_RES',
    'UVA_ACCESS', 
    'REPORT_CODE',
    'Need based',
    # 'PROGRAM_DIM_ID', # using ACADEMIC_PROGRAM_DESC instead since that has less unique counts
]

In [48]:
def subsets(columns):
    result = [[]]
    for column in columns:
        result += [subset + [column] for subset in result]
    return result

## Total party

In [ ]:
academic_groupby_key = [
    'ACADEMIC_LEVEL_TERM_START','ACADEMIC_PLAN',
    'ACADEMIC_PROGRAM_DESC'
]
enrolled_students = merge(
    student_term_fact, academic_program_dim
)
enrolled_students = merge(enrolled_students, term_dim)
# temp = temp[temp['TERM_TYPE'].isin(['Spring', 'Fall'])]

In [ ]:
print(f'Shape before {enrolled_students.shape}')
enrolled_students.drop_duplicates(['PROGRAM_DIM_ID', 'PARTY_DIM_ID', 'TERM_DIM_ID'], inplace=True)
print(f'Shape after dropping duplicates {enrolled_students.shape}')

In [ ]:
# update this list based on the previous groupby_key
total_party_by_group = enrolled_students.groupby(
    academic_groupby_key + [time_column]
)['PARTY_DIM_ID'].nunique().reset_index(name='TOTAL_PARTY')
total_party_by_group.groupby(time_column)['TOTAL_PARTY'].sum()

In [52]:
total_party_by_group = total_party_by_group[academic_groupby_key + [time_column] + ['TOTAL_PARTY']]

groups = []
for subset in subsets(academic_groupby_key):
    if len(subset) == len(academic_groupby_key): continue
    
    grouped = enrolled_students.groupby(
        subset + [time_column]
    )['PARTY_DIM_ID'].nunique().reset_index(name='TOTAL_PARTY')
    
    if len(subset)>0:
        temp = grouped.merge(total_party_by_group[academic_groupby_key], on=subset)
    
    for column in academic_groupby_key:
        if column in subset: continue
        # if len(subset) >0:
        #     if temp[column].nunique()==1: continue
            
        grouped[column] = 'Total'
    
    grouped = grouped[academic_groupby_key + [time_column] + ['TOTAL_PARTY']]
    groups.append(grouped)

groups = pd.concat(groups, axis=0)
total_party_by_group = pd.concat([total_party_by_group, groups], axis=0).reset_index(drop=True)

In [53]:
def drop_extra_total_rows(df, groupby_key):
    # the issue with the adding Total category is that when only one unique value 
    # is present in target column, adding Total aggregation is unnecessary. 
    # Which will have same target values since it is aggregated for one value. 
    # This is not what we want. The following code removes these rows
    print(f'Original size {df.shape}')
    
    for target_column in groupby_key:
        print(f'Processing {target_column}')
        groupby_without_target = [
            column for column in groupby_key if column != target_column
        ]
        
        others = []
        for subgroup_key, subgroup in tqdm(df.groupby(groupby_without_target)):
            values = subgroup[target_column].unique()
            if len(values) != 2 or 'Total' not in values:
                others.append(subgroup[groupby_key].drop_duplicates())
            else:
                temp = subgroup[groupby_key][subgroup[target_column] != 'Total']
                others.append(temp.drop_duplicates())
        
        others = pd.concat(others, axis=0).reset_index(drop=True)
        
        df = df.merge(others[groupby_key], on=groupby_key)
        print(f'Reduced {df.shape}')
        gc.collect()
    
    return df

In [ ]:
# total_party_by_group = drop_extra_total_rows(
#     total_party_by_group, academic_groupby_key
# )

In [ ]:
print(total_party_by_group.shape)
total_party_by_group = total_party_by_group.drop_duplicates(subset=academic_groupby_key + [time_column])
print(f'Shape after dropping duplicates {total_party_by_group.shape}')

In [57]:
total_party_by_group.to_csv(data_root + 'total_party_by_group.csv', index=False)

## Funded party

### Unique parties

In [58]:
grouped = df.groupby(groupby_key + [time_column])[[
    'INST_NEED', 'OFFER_BALANCE'
]]

funded_party = df.groupby(groupby_key + [time_column])[[
    'INST_NEED', 'OFFER_BALANCE'
]].sum().reset_index()

# how many unique parties got funded over a year
funded_party['FUNDED_PARTY'] = df.groupby(
    groupby_key + [time_column]
)['PARTY_DIM_ID'].nunique().values

In [59]:
# groupby key, time column, all other columns
columns = groupby_key + [time_column] + ['INST_NEED', 'OFFER_BALANCE', 'FUNDED_PARTY']
assert list(funded_party.columns) == columns,\
    f"Dataframe columns {funded_party.columns} do not match with rearranged {columns} columns"

funded_party = funded_party[columns]

### Add `Total` category
This is for dashboard view, when no single category is selected. For example, when no academic level is selected, default should show values aggregated for all levels. Hence we add a new label `Total` for such object features. This is also helpful for hierarchical forecasting, where we need aggreaged values at different hierarchies.

In [60]:
groups = []
for subset in subsets(groupby_key):
    if subset == groupby_key: continue
    
    grouped = df.groupby(subset + [time_column]
    )['PARTY_DIM_ID'].nunique().reset_index(name='FUNDED_PARTY')
    
    grouped[['INST_NEED', 'OFFER_BALANCE']] = df.groupby(
        subset + [time_column]
    )[['INST_NEED', 'OFFER_BALANCE']].sum().values
    
    if len(subset) > 0:
        temp = grouped.merge(funded_party[groupby_key], on=subset)
        
    for column in groupby_key:
        if column in subset: continue
        # if len(subset) >0:
        #     if temp[column].nunique()==1: continue
            
        grouped[column] = 'Total'
    
    # align columns for row-wise concat
    grouped = grouped[columns]
    groups.append(grouped)

groups = pd.concat(groups, axis=0)

In [ ]:
# groups = drop_extra_total_rows(groups, groupby_key)

In [62]:
groups.to_csv('datawarehouse/groups.csv', index=False)
# groups = pd.read_csv('datawarehouse/groups.csv')

In [ ]:
funded_party = pd.concat([funded_party, groups], axis=0).reset_index(drop=True)
print(funded_party.shape)

funded_party = funded_party.drop_duplicates(groupby_key + [time_column])
print(f'Shape after dropping duplicates {funded_party.shape}')

## Merge

In [ ]:
# add group id
ground_id = funded_party.groupby(groupby_key)[
    ['OFFER_BALANCE']
].sum().reset_index().reset_index(names='GROUP_ID')

ground_id.drop(columns='OFFER_BALANCE', inplace=True)
print(f'Group size {ground_id.shape}')

print(ground_id.head(3))

# add funded student info
funded_party = merge(ground_id, funded_party, key=groupby_key)

In [ ]:
# add student term file info, which includes all students, funded or not
# not that this info is only unique per the academic_groupby_key
# so summing by the groupby_key will count duplicates
summed = merge(funded_party, total_party_by_group)
summed = summed.drop_duplicates(subset=groupby_key + [time_column])

In [ ]:
# summed = drop_extra_total_rows(summed, groupby_key)

In [ ]:
# summed.groupby('GROUP_ID')[['COUNT', 'INST_NEED',	'OFFER_BALANCE']].sum().reset_index()
yearly = summed[(summed!='Total').all(axis=1)
    ].groupby(time_column)[['INST_NEED','OFFER_BALANCE']].sum().reset_index()
yearly

# Fill missing steps

For the time series analysis to work smoothly the different time series groups here needs to be of same length. Which it currently isn't because some funding may be available from 2014, when some might be introduced later. This section imputes those previous missing steps with 0, thus making all of this group lengths same. 

In [71]:
summed.to_csv(data_root + 'summed.csv', index=False)

In [ ]:
print(f'Shape before filling the missing timesteps {summed.shape}')

min_year, max_year = df[time_column].min(), df[time_column].max()
print(f'Min year {min_year}, max year {max_year}.')
years = pd.DataFrame(
    {time_column: range(min_year, max_year+1)}
)
merged = []

for group_key, group in tqdm(summed.groupby(groupby_key)):
    if group.shape[0] == 1:
        continue
    
    filled = years.merge(group, on=time_column, how='left')
    filled[groupby_key] = group_key
    filled['GROUP_ID'] = group['GROUP_ID'].values[0]
    merged.append(filled.fillna(0))

merged = pd.concat(merged, axis=0).reset_index(drop=True)
print(f'Shape after filling the missing timesteps {merged.shape}')

# Dump

In [ ]:
merged.head(5)

In [73]:
merged.to_csv(
    data_root + 'Merged2.csv', index=False
)

df.to_csv(
    data_root + 'Total2.csv', index=False
)